# PROJECT SUBMISSION A BY 
  * HARDIK MITTAL
  * SANKALP BAHAD
  * SUYASH SETHIA

### Google colab python notebook link : https://colab.research.google.com/drive/1GQTX4NUljZtXFM0bmc2Ajp6E-IPgtVJt?usp=sharing

## IMPORTING THE REQUIRED LIBRARIES

In [1]:
import re #importing regex package (for using the searching operation)
import sys #just added coz it was aded earlier as well :0

#from google.colab import files #to import/upload the teams_1.txt

import numpy as np #not required yet
import scipy as sio 
import pandas as pd #not required yet
import matplotlib.pyplot as plt #not required yet

#FOR THE STEMMER
!pip3 install spacy 
import spacy
from spacy.lang.hi import Hindi

#from wordsDict import *
!pip3 install stanza
import stanza
stanza.download('hi')


Defaulting to user installation because normal site-packages is not writeable


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defaulting to user installation because normal site-packages is not writeable


2022-06-17 13:16:02 INFO: Downloading default packages for language: hi (Hindi)...
2022-06-17 13:16:02 INFO: File exists: /home/hardk/stanza_resources/hi/default.zip
2022-06-17 13:16:03 INFO: Finished downloading models and saved to /home/hardk/stanza_resources.


## CLONING THE PROVIDED PARSER (ISCNLP)

In [2]:

## USE THESE COMMANDS MANUALLY IN THE TERMINAL
'''
!rm -r pos-tagger #so that at every call new postagger is created
!git clone https://irshadbhat@bitbucket.org/iscnlp/pos-tagger.git  #cloning the pos tagger as mentioned in the documentation of iscnlp

!cd pos-tagger
!sudo python3 setup.py install
!cd ..

!rm -r parser #so that everytime it clones again, can be removed too
!git clone https://irshadbhat@bitbucket.org/iscnlp/parser.git #to clone the parser 
!cd parser
!sudo python3 setup.py install #installing the hindi parser
!cd .. # to back to the main directorynlp = Hindi()
# doc = nlp(str)
# for token in doc:
#   print(token.text,"->", token.norm_)


#ls
'''

'\n!rm -r pos-tagger #so that at every call new postagger is created\n!git clone https://irshadbhat@bitbucket.org/iscnlp/pos-tagger.git  #cloning the pos tagger as mentioned in the documentation of iscnlp\n\n!cd pos-tagger\n!sudo python3 setup.py install\n!cd ..\n\n!rm -r parser #so that everytime it clones again, can be removed too\n!git clone https://irshadbhat@bitbucket.org/iscnlp/parser.git #to clone the parser \n!cd parser\n!sudo python3 setup.py install #installing the hindi parser\n!cd .. # to back to the main directorynlp = Hindi()\n# doc = nlp(str)\n# for token in doc:\n#   print(token.text,"->", token.norm_)\n\n\n#ls\n'

## IMPORTING LIBRAIRES FROM THE PROVIDED PARSER

In [3]:
from __future__ import unicode_literals
from isc_parser import Parser
parser = Parser(lang='hin')

## STEMMER

In [4]:
words_dict  = { "तैराक":"तैर",
              "चालाक":"चाल",
              "कूलाक":"कूल",
              "बेलन":"बेल",
              "मिलाप":"मिल",
              "चुपचाप": "चुप",
              "निकास":"निकस",
              "लुकास":"लुक",
              }

suffixes = {
    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],  
          2: ["तृ","ान","ैत","ने","ाऊ","ाव","कर", "ाओ", "िए", "ाई", "ाए", "नी", "ना", "ते", "ीं", "ती",
              "ता", "ाँ", "ां", "ों", "ें","ीय", "ति","या", "पन", "पा","ित","ीन","लु","यत","वट","लू"],     
          3: ["ेरा","त्व","नीय","ौनी","ौवल","ौती","ौता","ापा","वास","हास","काल","पान","न्त","ौना","सार","पोश","नाक",
              "ियल","ैया", "ौटी","ावा","ाहट","िया","हार", "ाकर", "ाइए", "ाईं", "ाया", "ेगी", "वान", "बीन",
              "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं","कला","िमा","कार",
              "गार", "दान","खोर"],     
          4: ["ावास","कलाप","हारा","तव्य","वैया", "वाला", "ाएगी", "ाएगा", "ाओगी", "ाओगे", 
              "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां",
              "त्वा","तव्य","कल्प","िष्ठ","जादा","क्कड़"],     
          5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां", "अक्कड़","तव्य:","निष्ठ"],
}

special_suffixes = ["र्", "ज्य","त्य"]
dict_special_suffixes = {"र्":"ृ",
                        "ज्य":"ज्",
                        "त्य":"त्"}

def hi_stem(word, clean=False,chars=None):
  if clean == True:
      word = clean_text(word, chars)
  
  ans = word
  bl = False
  
  if word in words_dict.keys():
      return words_dict[word]
  
  for L in 5, 4, 3, 2, 1:
      if len(word) > L + 1:
          for suf in suffixes[L]:
              if word.endswith(suf):
                  ans = word[:-L]
                  bl =True
      if bl == True:
          break
                  
  if bl == True:
      for suf in suffixes[1]:
          if ans.endswith(suf):
              # use case - गानेवाला
              ans = hi_stem(ans)
  
  for suf in special_suffixes:
      if ans.endswith(suf):
          l = len(suf)
          ans = ans[:-l]
          ans += dict_special_suffixes[suf]

  return ans

def clean_text(text, chars=None):
  if chars == None:        
      text = re.sub(r"[()\"#/@;:<>{}`+=~|!?,']", "", text)
  else:
      text = re.sub(r"[" +chars+ "()\"#/@;:<>{}`+=~|!?,']", "", text)
  return text



## Finds first verb in the text per line present just before the "कि" and stems it

In [5]:
output = open("output.txt", "w")
hi_nlp = stanza.Pipeline('hi')
c = 0  # line counter

pattern = " कि "  # added the spaces so that other words having "ki" are also not get used

with open("team_1.txt", "r") as file:  # opening the file team_1.txt to get its content
    for line in file:  # parsing the file line by line
        if re.search(pattern, line):  # if there is "ki" in the line
            k = "Sentence" + "<" + str(c) + ">" + "\n" + line +"\n"
            
            doc=hi_nlp(line)
            line_list = line.split()  # tokenize the sentence into words and stores in the list line
            line_list = [word for word in line_list if(word != "ं")]
            # parser makes list inside list for the parsed words
            parsed = parser.parse(line_list)

            
            word_before_ki = ""
            for word in parsed:
                
                if(word[2] == "कि"):
                    break
                if(word[3] == "VM"):
                    word_before_ki = word[2]

            if(word_before_ki != ""):
                output.write("----------------------------------\n")
                output.write(k)
                print("----------------------------------\n")
                print(k)
            
                word_before_ki=word_before_ki.replace("," , "") #just in case an extra, is added

                for sentence in doc.sentences:
                    for word in sentence.words:
                        
                        if (word.text==word_before_ki):
                            oupt=word.text+"->" +word.lemma+"\n"
                            output.write(oupt)
                            print(oupt) 
                            break           
        c=c+1  # to keep the track of the line numbers

2022-06-17 13:16:08 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |
| depparse  | hdtb    |

2022-06-17 13:16:08 INFO: Use device: gpu
2022-06-17 13:16:08 INFO: Loading: tokenize
/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA GeForce RTX 3060 Laptop GPU with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3060 Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.